In [28]:
import pandas as pd
import torch
import copy
import os

from sklearn.model_selection import train_test_split
from itertools import product
from sklearn.metrics import r2_score
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor


In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [30]:
#import dataset
df = pd.read_csv('dataset-ml-25m/dataset.csv')

In [31]:
# Dividi il dataset in feature e target
X = df.drop(['rating'], axis=1).to_numpy()
y = df['rating'].to_numpy()

# Dividi il dataset in training, validation e test
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
#count the numebr of x_train 
print("Number of train set: ", X_train.shape[0])
print("Numebr of test set: ", X_test.shape[0])
print("Number of validation set: ", X_val.shape[0])


Number of train set:  9946
Numebr of test set:  2764
Number of validation set:  1106


In [32]:
batch_sizes = [8, 16, 32]
n_epochs = [200]
#dimensionality of the output space of the feature transformer network
n_d = [8, 16, 32]
 #dimensionality of the output space of the attention network
n_a = [8, 16, 32]
#number of sequential steps in the attention mechanism 
n_steps = [3, 5, 7]
#number of independent networks
n_indipendent = [3, 5]
params = list(product(batch_sizes, n_epochs, n_d, n_a, n_steps, n_indipendent))
comb = len(batch_sizes)*len(n_d)*len(n_a) *len(n_steps) *len(n_indipendent)
print("Number of combinations: ", comb)

Number of combinations:  162


In [37]:
def get_model(batch_size, n_epochs, n_d, n_a, n_steps, n_indipendent):
    model = TabNetRegressor(
        n_d=n_d,
        n_a=n_a,
        n_steps=n_steps
    )
    return model

In [38]:
best_mse = float('inf')
best_model = None
best_params = None
iter = 0
for b, n_e, n_d, n_a, n_s, n_i in params:
    iter += 1
    print(f'\nIteration {iter}/{comb}')
    print(f"Configuration batch size: {b} - epochs: {n_e} - n_d: {n_d} - n_a: {n_a} - steps: {n_s} - n_indipendent: {n_i}")
    
    model = get_model(b, n_e, n_d, n_a, n_s, n_i)
    #save results for each iteration with tensorboard
    log = f'./results/TabNet/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}'

    if os.path.exists("./results/TabNet"+log):
        print("Model already trained. Skipping...")
        continue

    writer = SummaryWriter(f'results/TabNet/bach_size_{b}_nEpochs_{n_e}_nd_{n_d}_na_{n_a}_nSteps_{n_s}_nIndipendent_{n_i}')
    #fit model
    model.fit(
        X_train=X_train,
        y_train=y_train,
        eval_set=[(X_val, y_val)],
        eval_name=['mse'],
        patience=10,
        batch_size=b
    )

    # evaluate model
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    #save mse and r2 score for each iteration with tensorboard
    writer.add_scalar('MSE', mse)
    writer.add_scalar('R2 Score', r2)
    print('MSE:', mse)
    print('R2 Score:', r2)
    if mse < best_mse:
        best_mse = mse
        best_model = copy.deepcopy(model)
        best_params = (b, n_e, n_d, n_a, n_s, n_i)
        print('Best model updated')
    writer.close()
            


Iteration 1/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 3


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.2812  | mse_mse: 0.07043 |  0:00:21s
epoch 1  | loss: 0.09894 | mse_mse: 0.27305 |  0:00:42s
epoch 2  | loss: 0.0819  | mse_mse: 0.07393 |  0:01:04s
epoch 3  | loss: 0.07828 | mse_mse: 0.06278 |  0:01:26s
epoch 4  | loss: 0.07167 | mse_mse: 0.05142 |  0:01:47s
epoch 5  | loss: 0.07409 | mse_mse: 0.06293 |  0:02:09s
epoch 6  | loss: 0.07251 | mse_mse: 0.06103 |  0:02:31s
epoch 7  | loss: 0.06916 | mse_mse: 0.0606  |  0:02:53s
epoch 8  | loss: 0.06491 | mse_mse: 0.0412  |  0:03:16s
epoch 9  | loss: 0.0633  | mse_mse: 0.05938 |  0:03:37s
epoch 10 | loss: 0.06093 | mse_mse: 0.10407 |  0:04:00s
epoch 11 | loss: 0.06202 | mse_mse: 0.06393 |  0:04:22s
epoch 12 | loss: 0.0585  | mse_mse: 0.07598 |  0:04:44s
epoch 13 | loss: 0.05858 | mse_mse: 0.07784 |  0:05:06s
epoch 14 | loss: 0.06011 | mse_mse: 0.04215 |  0:05:28s
epoch 15 | loss: 0.05708 | mse_mse: 0.04663 |  0:05:50s
epoch 16 | loss: 0.05288 | mse_mse: 0.0658  |  0:06:12s
epoch 17 | loss: 0.04876 | mse_mse: 0.05388 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.03906676625632957
R2 Score: 0.8237584937093532
Best model updated

Iteration 2/162
Configuration batch size: 8 - epochs: 200 - n_d: 8 - n_a: 8 - steps: 3 - n_indipendent: 5


...

MSE: 0.014322879234792673
R2 Score: 0.9353853402097261

Iteration 162/162
Configuration batch size: 32 - epochs: 200 - n_d: 32 - n_a: 32 - steps: 7 - n_indipendent: 5


/usr/local/lib/python3.9/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.947   | mse_mse: 0.61709 |  0:00:16s
epoch 1  | loss: 0.16289 | mse_mse: 0.15501 |  0:00:32s
epoch 2  | loss: 0.12447 | mse_mse: 0.07389 |  0:00:49s
epoch 3  | loss: 0.09397 | mse_mse: 0.06228 |  0:01:06s
epoch 4  | loss: 0.0752  | mse_mse: 0.0422  |  0:01:23s
epoch 5  | loss: 0.07073 | mse_mse: 0.06951 |  0:01:40s
epoch 6  | loss: 0.07103 | mse_mse: 0.04148 |  0:01:58s
epoch 7  | loss: 0.05323 | mse_mse: 0.04399 |  0:02:15s
epoch 8  | loss: 0.05151 | mse_mse: 0.03945 |  0:02:32s
epoch 9  | loss: 0.04488 | mse_mse: 0.024   |  0:02:49s
epoch 10 | loss: 0.03314 | mse_mse: 0.02087 |  0:03:06s
epoch 11 | loss: 0.03776 | mse_mse: 0.04919 |  0:03:25s
epoch 12 | loss: 0.03407 | mse_mse: 0.01274 |  0:03:42s
epoch 13 | loss: 0.03489 | mse_mse: 0.04426 |  0:03:59s
epoch 14 | loss: 0.03635 | mse_mse: 0.0225  |  0:04:16s
epoch 15 | loss: 0.03413 | mse_mse: 0.02547 |  0:04:34s
epoch 16 | loss: 0.03653 | mse_mse: 0.04732 |  0:04:51s
epoch 17 | loss: 0.03363 | mse_mse: 0.01634 |  0

/usr/local/lib/python3.9/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


MSE: 0.014322879234792673
R2 Score: 0.9353853402097261


In [39]:
#pint the best model
print(best_model)
print("Best MSE: ", best_mse)
print("R2 score", r2_score(y_test, best_model.predict(X_test)))


TabNetRegressor(n_d=16, n_a=32, n_steps=7, gamma=1.3, cat_idxs=[], cat_dims=[], cat_emb_dim=1, n_independent=2, n_shared=2, epsilon=1e-15, momentum=0.02, lambda_sparse=0.001, seed=0, clip_value=1, verbose=1, optimizer_fn=<class 'torch.optim.adam.Adam'>, optimizer_params={'lr': 0.02}, scheduler_fn=None, scheduler_params={}, mask_type='sparsemax', input_dim=1128, output_dim=1, device_name='auto', n_shared_decoder=1, n_indep_decoder=1)
Best MSE:  0.00639599107305163
R2 score 0.9711458303577016
